In [70]:
%pip install mne pingouin pandas Path ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [71]:
import mne
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel
from ipywidgets import interact, Dropdown

## Viewing EEG

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

bands     = ["Delta", "Theta", "Alpha", "Beta", "Gamma"]
channels  = ["TP9", "AF7", "AF8", "TP10"]
band_cols = [f"{b}_{ch}" for b in bands for ch in channels]


In [73]:
def load_clean(path):
    df = pd.read_csv(path)

    # Remove Muse event rows like "/muse/event/connected …"
    if "Elements" in df.columns:
        df = df[~df["Elements"].astype(str).str.startswith("/muse/event")]

    # Keep only columns that exist in this file
    cols = [c for c in band_cols if c in df.columns]
    return df[cols]



In [ ]:
pre = load_clean("12-48-54_first5min.csv")  # post-task file
post  = load_clean("13-23-51_first5min.csv")   # baseline file

pre = load_clean("../session_data/")  # post-task file
post  = load_clean("../session_data/")   # baseline file

C:\Users\rm534\AppData\Local\Temp\ipykernel_29420\3175224350.py:2: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\rm534\AppData\Local\Temp\ipykernel_29420\3175224350.py:2: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


### View Head of Data

In [77]:

# Make sure we use the intersection of columns present in both
common_cols = sorted(set(pre.columns) & set(post.columns))
pre  = pre[common_cols]
post = post[common_cols]

print(pre.shape, post.shape)
pre.head()

(77330, 20) (619, 20)


,Alpha_AF7,Alpha_AF8,Alpha_TP10,Alpha_TP9,Beta_AF7,Beta_AF8,Beta_TP10,Beta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Delta_TP9,Gamma_AF7,Gamma_AF8,Gamma_TP10,Gamma_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Theta_TP9
1,0.468047,0.464054,0.874555,0.98637,0.319931,-0.032291,0.640663,0.674551,1.008486,1.000646,1.329328,1.526259,0.050519,-0.13238,0.070085,0.227895,0.538246,0.640503,1.192411,1.476564
2,0.468047,0.464054,0.874555,0.98637,0.319931,-0.032291,0.640663,0.674551,1.008486,1.000646,1.329328,1.526259,0.050519,-0.13238,0.070085,0.227895,0.538246,0.640503,1.192411,1.476564
3,0.468047,0.464054,0.874555,0.98637,0.319931,-0.032291,0.640663,0.674551,1.008486,1.000646,1.329328,1.526259,0.050519,-0.13238,0.070085,0.227895,0.538246,0.640503,1.192411,1.476564
4,0.468047,0.464054,0.874555,0.98637,0.319931,-0.032291,0.640663,0.674551,1.008486,1.000646,1.329328,1.526259,0.050519,-0.13238,0.070085,0.227895,0.538246,0.640503,1.192411,1.476564
5,0.468047,0.464054,0.874555,0.98637,0.319931,-0.032291,0.640663,0.674551,1.008486,1.000646,1.329328,1.526259,0.050519,-0.13238,0.070085,0.227895,0.538246,0.640503,1.192411,1.476564


### Summary Table of Bands

In [78]:
pre_means  = pre.mean()
post_means = post.mean()

summary = pd.DataFrame({
    "pre":  pre_means,
    "post": post_means,
})
summary["diff"] = summary["post"] - summary["pre"]

# split "Alpha_AF8" into band + channel for convenience
summary["band"]    = summary.index.str.split("_").str[0]
summary["channel"] = summary.index.str.split("_").str[1]

summary = summary.sort_values("diff", ascending=False)
summary

,pre,post,diff,band,channel
Gamma_AF8,-0.675175,-0.288885,0.386290,Gamma,AF8
Gamma_AF7,-0.537031,-0.344782,0.192249,Gamma,AF7
Beta_AF8,-0.277113,-0.134929,0.142184,Beta,AF8
Gamma_TP9,-0.307041,-0.207998,0.099043,Gamma,TP9
Gamma_TP10,-0.275445,-0.205341,0.070104,Gamma,TP10
Delta_AF7,-0.126627,-0.061446,0.065181,Delta,AF7
Beta_AF7,-0.202826,-0.140982,0.061844,Beta,AF7
Beta_TP9,0.420394,0.481906,0.061512,Beta,TP9
Beta_TP10,0.500086,0.532168,0.032082,Beta,TP10
Alpha_AF8,-0.106035,-0.090049,0.015986,Alpha,AF8


In [79]:
def plot_eeg_view(view, band, channel):
    plt.figure(figsize=(8, 4))

    if view == "All bands & channels":
        sub = summary.copy()
        x = range(len(sub))
        width = 0.4

        plt.bar([i - width/2 for i in x], sub["pre"],  width=width, label="pre")
        plt.bar([i + width/2 for i in x], sub["post"], width=width, label="post")

        plt.xticks(x, sub.index, rotation=90)
        plt.ylabel("Mean band power (log)")
        plt.title("Pre vs Post – all band×channel")

    elif view == "One band across channels":
        sub = summary[summary["band"] == band].copy()
        sub = sub.sort_values("channel")  # tidy order

        x = range(len(sub))
        width = 0.35

        plt.bar([i - width/2 for i in x], sub["pre"],  width=width, label="pre")
        plt.bar([i + width/2 for i in x], sub["post"], width=width, label="post")

        plt.xticks(x, sub["channel"])
        plt.ylabel("Mean band power (log)")
        plt.title(f"Pre vs Post – {band} across channels")

    elif view == "One channel across bands":
        sub = summary[summary["channel"] == channel].copy()
        # keep a nice band order
        order = [b for b in ["Delta", "Theta", "Alpha", "Beta", "Gamma"] if b in sub["band"].values]
        sub = sub.set_index("band").loc[order]

        x = range(len(sub))
        width = 0.35

        plt.bar([i - width/2 for i in x], sub["pre"],  width=width, label="pre")
        plt.bar([i + width/2 for i in x], sub["post"], width=width, label="post")

        plt.xticks(x, sub.index)
        plt.ylabel("Mean band power (log)")
        plt.title(f"Pre vs Post – {channel} across bands")

    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
interact(
    plot_eeg_view,
    view=Dropdown(
        options=[
            "All bands & channels",
            "One band across channels",
            "One channel across bands",
        ],
        value="One band across channels",
        description="View:"
    ),
    band=Dropdown(
        options=bands,
        value="Alpha" if "Alpha" in bands else bands[0],
        description="Band:"
    ),
    channel=Dropdown(
        options=channels,
        value="AF8" if "AF8" in channels else channels[0],
        description="Channel:"
    )
);

interactive(children=(Dropdown(description='View:', index=1, options=('All bands & channels', 'One band across…

What we are looking for: 
(IEEE)
- increase in cognitive fatigue (AF8 High Alpha) and a   
-  increase of cognitive workload (AF8 Low Beta).

### EEG Kolmogorov Smirnov test


- test whether two underlying one-dimensional probability distributions differ
- averaging over 5 seconds 

comparing outputs - 

rejected is a boolean array telling you which tests remain significant.

In [81]:
import numpy as np
from statsmodels.stats.multitest import multipletests

# Benjamini-Hichberg FDR at alpha -



# Benjamini-Hochberg FDR at alpha=0.05
# rejected, pcorrected, __build_class__ = multipletests(pvals, alpha=0.05, method='fdr_bh')

# print("Significant after FDR:", rejected)
# print("Corrected p-values:", p_corrected)
# pvals = np.array([... your 16 p-values ...])


#### Kolomogorov-Smirnov Test 

In [82]:
# average into 5 seconds - EEG change rate of 
# pre[""]

#### Man-Whitney U test

#### Benjamini-Hichberg FDR